# Hatch Template!

## Dandelion Voting

1. Percentage of total tokens that have to vote 'yes' to `something` for it to pass.

In [27]:
import param
import panel as pn
import pandas as pd
import hvplot.pandas
import holoviews as hv
pn.extension()

In [ ]:
class DandelionVoting(param.Parameterized):
    total_tokens = param.Number(1e6, constant=True)
    support_required = param.Number(0.5, bounds=(0.5,0.9), step=0.01) # Of those who voted, this percent must be yes to pass
    minimum_quorum = param.Number(0.02, bounds=(0,1), step=0.01)
    

In [28]:
class DandelionVoting(param.Parameterized):
    total_tokens = param.Number(1e6, constant=True)
    minimum_quorum = param.Number(0.02, bounds=(0,1), step=0.01)
    support_required = param.Number(0.5, bounds=(0.5,0.9), step=0.01)
    days_to_vote_on_proposal = param.Integer(3 + 8 + 24, bounds=(0,100))
    days_to_exit_hatch = param.Integer(8)
#     vote_buffer_blocks = param.Integer(8, bounds=(0,None))
#     vote_execution_delay_blocks = param.Integer(24, bounds=(0,None))
    cost_to_make_a_proposal = param.Number(3, step=1, doc="cost to make a proposal")
    maximum_number_proposals_per_month = param.Number(10, bounds=(1, 100))
    
    def view(self):
        min_yes_tokens = self.support_required * self.minimum_quorum * self.total_tokens
        min_blockers = (1 - self.support_required) * self.minimum_quorum * self.total_tokens
        votes = pd.DataFrame.from_dict({'Votes': [min_yes_tokens, min_blockers]}, orient='index', columns=['Minimum Tokens to Pass', 'Minimum Tokens for Quorum'])
        vote_plot = votes.hvplot.bar(stacked=True, ylim=(0,self.total_tokens)).opts(color=hv.Cycle(['#0F2EEE', '#0b0a15', '#DEFB48']))
        return pn.Row(vote_plot, pn.Column("Minimum Tokens to Meet Quorum: ", int(self.minimum_quorum * self.total_tokens), "Minimum Tokens to Pass a Vote: ", int(min_yes_tokens), "Minimum Tokens to Block a Vote: ", int(min_blockers)))

In [29]:
d = DandelionVoting()

In [36]:
pn.Column(d, d.view)

Column
    [0] Column(margin=5, name='DandelionVoting', width=300)
        [0] StaticText(value='<b>DandelionVoting</b>')
        [1] FloatInput(disabled=True, name='Total tokens', value=1000000.0, value_throttled=1000000.0)
        [2] FloatSlider(name='Minimum quorum', step=0.01, value=0.02, value_throttled=0.02)
        [3] FloatSlider(end=0.9, name='Support required', start=0.5, step=0.01, value=0.5, value_throttled=0.5)
        [4] IntSlider(end=100, name='Days to vote o..., value=35, value_throttled=35)
        [5] IntInput(name='Days to exit hatch', value=8, value_throttled=8)
        [6] FloatInput(name='Cost to make a proposal', step=1, value=3, value_throttled=3)
        [7] FloatSlider(end=100, name='Maximum number p..., start=1, value=10, value_throttled=10)
    [1] ParamMethod(method)

In [37]:
class Hatch(param.Parameterized):
    # CSTK Ratio
    total_cstk_tokens = param.Number(700000, constant=True)
    hatch_oracle_ratio = param.Number(0.005, constant=True)
    @param.depends('hatch_oracle_ratio', 'total_cstk_tokens')
    def wxdai_range(self):
        return pn.Row(pn.Pane("Cap on wxdai staked: "), self.hatch_oracle_ratio * self.total_cstk_tokens)
    
    # Min and Target Goals
    min_goal = param.Number(5, bounds=(1,100), step=10)
    max_goal = param.Number(1000, bounds=(100,10000), step=50) # Something to consider -> target goal or max goal
    
    # Hatch params
    hatch_period = param.Integer(15, bounds=(5, 30), step=2)
    hatch_exchange_rate = param.Number() # This needs to be tested and explained -> See the forum post
    hatch_tribute = param.Number(0.05, bounds=(0,1))    

In [38]:
h = Hatch()

pn.Pane(h)

Param(Hatch)

In [42]:
import pandas as pd
import panel as pn
import os
import hvplot.pandas
APP_PATH = './'

sheets = [
    "Total Impact Hours so far", 
    "IH Predictions", 
    "#8 Jan 1",
    "#7 Dec 18",
    "#6 Dec 4",
    "#5 Nov 20",
    "#4 Nov 6",
    "#3 Oct 23",
    "#2 Oct 9",
    "#1 Sept 24",
    "#0 Sept 7 (historic)",
] + [f"#{i} IH Results" for i in range(9)]
sheets = {i:sheet for i, sheet in enumerate(sheets)}

def read_excel(sheet_name="Total Impact Hours so far", header=1, index_col=0, usecols=None) -> pd.DataFrame:
    data = pd.read_excel(
        os.path.join(APP_PATH, "data", "TEC Praise Quantification.xlsx"),
        sheet_name=sheet_name,
        engine='openpyxl',
        header=header,
        index_col=index_col,
        usecols=usecols
    ).reset_index().dropna(how='any')
    return data

## Tests
total_impact_hours = read_excel()
impact_hour_data = read_excel(sheet_name="IH Predictions", header=0, index_col=0, usecols='A:I').drop(index=19)
pn.Row(impact_hour_data.hvplot.table(), total_impact_hours.hvplot.table())

Row
    [0] HoloViews(Table)
    [1] HoloViews(Table)

In [43]:
import numpy as np


class ImpactHours(param.Parameterized):
    max_ih_rate = param.Number(0.01, bounds=(0,200))
    expected_raise_per_ih = param.Number(0.012, bounds=(0,20))

    @param.depends('max_ih_rate', 'expected_raise_per_ih')
    def impact_hours_rewards(self):
        x = np.linspace(h.min_goal, h.max_goal)

        R = self.max_ih_rate

        m = self.expected_raise_per_ih
        
        H = total_impact_hours['Impact Hours'].sum()

        y = [R* (x / (x + m*H)) for x in x]

        df = pd.DataFrame([x,y]).T
        df.columns = ['x','y']

        return df.hvplot(x='x')

In [44]:
i = ImpactHours()

pn.Row(i, i.impact_hours_rewards)

Row
    [0] Column(margin=5, name='ImpactHours', width=300)
        [0] StaticText(value='<b>ImpactHours</b>')
        [1] FloatSlider(end=200, name='Max ih rate', value=0.01, value_throttled=0.01)
        [2] FloatSlider(end=20, name='Expected raise per ih', value=0.012, value_throttled=0.012)
    [1] ParamMethod(method)

In [46]:
pn.Row(d, h, i)

Row
    [0] Column(margin=5, name='DandelionVoting', width=300)
        [0] StaticText(value='<b>DandelionVoting</b>')
        [1] FloatInput(disabled=True, name='Total tokens', value=1000000.0, value_throttled=1000000.0)
        [2] FloatSlider(name='Minimum quorum', step=0.01, value=0.02, value_throttled=0.02)
        [3] FloatSlider(end=0.9, name='Support required', start=0.5, step=0.01, value=0.5, value_throttled=0.5)
        [4] IntSlider(end=100, name='Days to vote o..., value=35, value_throttled=35)
        [5] IntInput(name='Days to exit hatch', value=8, value_throttled=8)
        [6] FloatInput(name='Cost to make a proposal', step=1, value=3, value_throttled=3)
        [7] FloatSlider(end=100, name='Maximum number p..., start=1, value=10, value_throttled=10)
    [1] Column(margin=5, name='Hatch', width=300)
        [0] StaticText(value='<b>Hatch</b>')
        [1] FloatInput(disabled=True, name='Total cstk tokens', value=700000, value_throttled=700000)
        [2] FloatInput(disabled=True, name='Hatch oracle ratio', value=0.005, value_throttled=0.005)
        [3] FloatSlider(end=100, name='Min goal', start=1, step=10, value=5, value_throttled=5)
        [4] FloatSlider(end=10000, name='Max goal', start=100, step=50, value=1000, value_throttled=1000)
        [5] IntSlider(end=30, name='Hatch period', start=5, step=2, value=15, value_throttled=15)
        [6] FloatInput(name='Hatch exchange rate', value_throttled=0.0)
        [7] FloatSlider(name='Hatch tribute', value=0.05, value_throttled=0.05)
    [2] Column(margin=5, name='ImpactHours', width=300)
        [0] StaticText(value='<b>ImpactHours</b>')
        [1] FloatSlider(end=200, name='Max ih rate', value=85, value_throttled=85)
        [2] FloatSlider(end=20, name='Expected raise per ih', value=1.3, value_throttled=1.3)

In [47]:
pn.Row(d.view, i.impact_hours_rewards)

Row
    [0] ParamMethod(method)
    [1] ParamMethod(method)

# Target/Expected Goals

In [ ]:
class CommunityParticipation(param.Parameterized)